# Preparing wsd datasets

In [1]:
import requests
import MySQLdb
import json
# _db = MySQLdb.connect(host="127.0.0.1",port=3307,user='root',passwd="emilios",db="enwiki20160305")
# _cursor = _db.cursor()
# _cursor.execute("""SELECT * FROM `anchors` group""", (wid,))


In [2]:
%load_ext autoreload
%autoreload


import re
import json
import sys
import urllib
import sys
import requests
import MySQLdb

from HTMLParser import HTMLParser
sys.path.append('../../cgi-bin/')
%aimport wikipedia
from wikipedia import *

def qualify(r, min_mention=2, min_abmig=2):
    
    # condition 1
    if len(r['opening_annotation']) < min_mention:
        #print "c1 failed"
        return None
    
    nottop = False
    found_min_ambig=False
    annotations = json.loads(r['opening_annotation'])
    for ann in annotations:
        surf = ann['surface_form']
        linktitle = ann['url']
        wid = title2id(linktitle.encode('utf-8').replace(' ','_'))
        # condition 2
        if wid is None:
            #print "c2 failed", linktitle
            return None
        
        allids = anchor2concept(surf)
        # condition 3
        if not allids:
            #print "c3 failed"
            return None
        # condition 4
        if len(allids) >= min_abmig:
            found_min_ambig = True
        allids = sorted(allids, key=lambda k:-k[1])
        
        # condition 5
        #print allids
        if allids[0][0] != wid:
            nottop = True
    if not found_min_ambig:
        #print "c4 failed"
        return None
    return nottop                                                                                   
    
    
    
    

In [3]:
#test qualify
#Conference
#Oladapo
#r={'opening_annotation':[{'url':'Bruce_Lee', 'surface_form': 'Bruce Lee'}, {'url':'Sanandaj', 'surface_form': 'Sanandaj'}]}
#r={'opening_annotation':[{'url':'Oladapo', 'surface_form': 'Oladapo'}, {'url':'Do_while_loop', 'surface_form': 'do while loop'}]}
#r={'opening_annotation':[{'url':'David_Beckham', 'surface_form': 'David'}, {'url':'Do_while_loop', 'surface_form': 'do while loop'}]}
#print qualify(r)
#print len(anchor2concept('Bruce'))

In [5]:
wanted=5000
wanted_nottop=1000

reopen()
qstr = 'http://localhost:8983/solr/enwiki20160305/select'
q='*:*'
params={'indent':'on', 'wt':'json', 'fl':'title opening_text opening_annotation', 'q':q, "start":0,
        "rows":100}
home = os.path.expanduser("~");
output = open(os.path.join(home,'backup/datasets/ner/wiki.5000.json'), 'w')

start=0
count=0
count_nottop=0
enough=False
while True:
    params["start"] = start
    start += 1
    r = requests.get(qstr, params=params)
    #print r.json()['response']['docs']
    for d in r.json()['response']['docs']:
        #print d["id"]; enough=True; break
        if not (count < wanted or count_nottop < wanted_nottop):
            enough = True
            break
        qd = qualify(d) 
        if qd is None:
            continue
            
        if count_nottop < wanted_nottop and qd :
            output.write(json.dumps(d, ensure_ascii=False).encode('utf-8')+'\n')
            count_nottop += 1
            count += 1
            if ((count) % int(wanted/10) ==0) or (count_nottop) % int(wanted_nottop/10) ==0:
                print "count: ", count, ", count_nottop: ", count_nottop
            continue
        if count < wanted:
            count +=1
            output.write(json.dumps(d, ensure_ascii=False).encode('utf-8')+'\n')
            if (count) % int(wanted/10) ==0:   
                print "count: ", count
            
    #break
    if enough:
        break
print 'done'        
output.close()    
#r=json.loads(r['opening_annotation'])


count:  314 , count_nottop:  100
count:  500
count:  689 , count_nottop:  200
count:  1000
count:  1201 , count_nottop:  300
count:  1500
count:  1584 , count_nottop:  400
count:  1957 , count_nottop:  500
count:  2000
count:  2253 , count_nottop:  600
count:  2500
count:  2511 , count_nottop:  700
count:  2785 , count_nottop:  800
count:  3000
count:  3034 , count_nottop:  900
count:  3262 , count_nottop:  1000
count:  3500
count:  4000
count:  4500
count:  5000
done


## Converting json to input format

In [6]:
import sys, os, json
import string
home = os.path.expanduser("~");
jsname = os.path.join(home,'backup/datasets/ner/wiki.5000.json')
outjsname = os.path.join(home,'backup/datasets/ner/wiki-mentions.5000.json')


    
def splittext(text,url):
    start=0
    termindex=0
    t=[]
    mentions=[]
    # pass 1, adjust partial mentions. 
    # approach one, expand (the other could be shrink)
    
    for u in url:
        seg = text[start:u['from']]
        t += seg.strip().split()
        mentions.append([len(t),u['url']])
        t+=[" ".join(text[u['from']:u['to']].split())]
        start = u['to']
        
    t += text[start:].strip().split()
    return t, mentions

with open(jsname) as jf, open(outjsname,'w') as outjs:
    for line in jf:
        line = line.strip().decode('utf-8')
        js = json.loads(line)
        text = js['opening_text'] 
        url = json.loads(js['opening_annotation'])
        t, m = splittext(text, url)
        outjs.write((json.dumps({"text":t, "mentions":m}, ensure_ascii=False)+'\n').encode('utf-8'))
print "done"

done


## Converting KORE to input format

In [ ]:
%load_ext autoreload
%autoreload

import sys, os, json
import string
from HTMLParser import HTMLParser
hp = HTMLParser()
home = os.path.expanduser("~");

sys.path.append('../../cgi-bin/')
%aimport wikipedia
from wikipedia import *
reopen()


infname = os.path.join(home,'backup/datasets/ner/KORE50/AIDA.tsv')
outjsname = os.path.join(home,'backup/datasets/ner/kore.json')
url_col = 3

t=[]
m=[]
with open(infname) as inf, open(outjsname,'w') as outjs:
    state = 0
    for line in inf:
        line = line.strip()
        if not line:
            continue
        try:
            line = line.decode('unicode-escape')
        except:
            pass            
        if line.startswith('-DOCSTART-'):
            if m and t:
                outjs.write (json.dumps({"text":t, "mentions":m}, ensure_ascii=False).encode('utf-8')+'\n')
            t = []
            m = []
            continue
                
        w = line.split('\t')
        if len(w) == 1:
            t.append(w[0])
            continue
            
        if w[1] == 'B':
            if title2id(w[3]) is None:
                print w[3], " not found"
            if w[url_col] != '--NME--' and (title2id(w[3]) is not None):
                m.append((len(t), w[3]))
            t.append(w[2])
            
        if w[1] == 'I':
            continue
    if m and t:
        outjs.write (json.dumps({"text":t, "mentions":m}, ensure_ascii=False).encode('utf-8')+'\n')
        
            



In [ ]:
x="Rudi_V\u00f6ller"
print x.decode('unicode-escape')

# Converting Aida

In [ ]:
%load_ext autoreload
%autoreload


import sys, os, json
import string
import re

from HTMLParser import HTMLParser
hp = HTMLParser()
home = os.path.expanduser("~");

sys.path.append('../../cgi-bin/')
%aimport wikipedia
from wikipedia import *
reopen()

pattern = re.compile("http://en.wikipedia.org/wiki/(.*)")


infname = os.path.join(home,'backup/datasets/ner/Aida01/aida-yago2-dataset/AIDA-YAGO2-annotations.tsv')
#infname = os.path.join(home,'backup/datasets/ner/aida-shalam.tsv')
outjsname = os.path.join(home,'backup/datasets/ner/aida.json')
#outjsname = os.path.join(home,'backup/datasets/ner/aida-shalam.json')
# set it to 3 for AIDA, 
url_col = 4
pattern = re.compile("http://en.wikipedia.org/wiki/(.*)")

t=[]
m=[]
with open(infname) as inf, open(outjsname,'w') as outjs:
    state = 0
    for line in inf:
        line = line.strip().decode('utf-8')
        #line = line.strip()
        if not line:
            continue
        if line.startswith('-DOCSTART-'):
            if m and t:
                outjs.write (json.dumps({"text":t, "mentions":m}, ensure_ascii=False).encode('utf-8')+'\n')
                #print (json.dumps({"text":t, "mentions":m}, ensure_ascii=False).encode('utf-8')+'\n')
            t = []
            m = []
            continue
                
        w = line.split('\t')
            
        if w[1] == '--NME--':
            continue
        title = w[2]
        r = re.match(pattern, title)
        title = r.group(1)
        rwd = title2id(title)
            
        rwd = title2id(title)
            
        anchors = id2anchor(rwd)
        if not anchors:
            print "no anchor for: ", title
            continue
                            
        c=max(anchors, key=lambda k:k[1])[0]    
        m.append((len(t), title))
        t.append(c.decode('utf-8'))
            
    if m and t:
        outjs.write (json.dumps({"text":t, "mentions":m}, ensure_ascii=False).encode('utf-8')+'\n')
        #print (json.dumps({"text":t, "mentions":m}, ensure_ascii=False).encode('utf-8')+'\n')
                
print "done"        
        
            



In [ ]:
%%bash
ls /home/sajadi/backup/archived/datasets/ner/shalam.tsv

In [ ]:
id2title(12633520)

In [ ]:
id2anchor(12633520)